# Speedscope Samples App

> Run all the cells to see the demo. Not all of the samples will work.
>
> Use the _Create New View For Output_ context menu on the output below for
a new tab 

In [ ]:
import sys

if __name__ == "__main__" and "pyodide" in sys.modules:
    %pip install -r requirements.txt

In [ ]:
import ipywidgets as W

app = W.VBox(layout={"height": "100%", "min_height": "60vh"})
app

In [ ]:
import tarfile
from pathlib import Path
from urllib.request import urlretrieve

import traitlets as T

from ipyprofiler import Flamegraph

In [ ]:
URL = "https://github.com/jlfwong/speedscope/archive/refs/tags/v1.20.0.tar.gz"

Build a function which can load samples.

In [ ]:
def get_samples():
    """Fetch the upstream tarball and load the samples."""
    samples = {}
    tarball = Path(Path(URL).name)
    if not tarball.exists():
        urlretrieve(URL, tarball)
    with tarfile.open(tarball) as tf:
        for info in tf.getmembers():
            if info.isdir() or "sample/profiles" not in info.name:
                continue
            label = info.name.split("sample/profiles/")[-1].replace("/", ": ")
            samples[label] = tf.extractfile(info).read()
    return samples

Fetch and initialize the samples.

In [ ]:
samples = get_samples()

Actually create the widgets.

In [ ]:
fg = Flamegraph()
select = W.Select(description="sample", options=sorted(samples.keys()), rows=1)
T.dlink((select, "value"), (fg.profile, "value"), lambda k: samples[k])
app.children = [select, fg]